### Import Packages

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
%matplotlib inline
import seaborn as sns
print("libraries imported successfully")

libraries imported successfully


### Get The Data

In [2]:
datatype={'fare_amount':'float64','pickup_datetime':'object', 'pickup_longitude':'float64', 'pickup_latitude':'float64',
       'dropoff_longitude':'float64', 'dropoff_latitude':'float64', 'passenger_count':'int64'}

In [3]:
df = pd.read_csv('EDA_data.csv',dtype = datatype,parse_dates=['pickup_datetime'])

### Split The Data

In [4]:
X= df.drop(['fare_amount','pickup_datetime','Unnamed: 0'],axis=1)
Y= df[['fare_amount']]

In [5]:
X.columns,Y.columns

(Index(['pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
        'dropoff_latitude', 'passenger_count', 'year', 'month', 'day',
        'weekday', 'pickup_datetime_hour', 'distance_km'],
       dtype='object'),
 Index(['fare_amount'], dtype='object'))

In [6]:
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test = train_test_split(X,Y, test_size=0.3, random_state=21)
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape

((4387714, 11), (1880449, 11), (4387714, 1), (1880449, 1))

### Build The Model

In [7]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [8]:
def eval_fun(model,X_train,Y_train,Y_test,X_test):

    Y_pred_train = model.predict(X_train)
    Y_pred_test = model.predict(X_test)
    
    r2_train = r2_score(Y_train,Y_pred_train)
    mse_train = mean_squared_error(Y_train,Y_pred_train)
    mae_train = mean_absolute_error(Y_train,Y_pred_train)
    print(f"r2_train_score ==> {r2_train} , mse_train_score ==> {mse_train} , mae_train_score ==> {mae_train}")
  
    r2_test = r2_score(Y_test,Y_pred_test)
    mse_test = mean_squared_error(Y_test,Y_pred_test)
    mae_test = mean_absolute_error(Y_test,Y_pred_test)
    print(f"r2_test_score ==> {r2_test} , mse_test_score ==> {mse_test} , mae_test_score ==> {mae_test}")

In [9]:
from xgboost import XGBRegressor 
from sklearn.model_selection import GridSearchCV
import pickle

In [10]:
xgb = XGBRegressor(random_state=21)

In [11]:
param_grid = {
'n_estimators' : [30, 50, 70],
'max_depth' :[9,12,14],
'learning_rate':[0.2,0.3]
}

In [12]:
gscv =GridSearchCV(estimator = xgb, param_grid = param_grid, cv=2, verbose=2)

In [13]:
gscv.fit(X_train,Y_train)
gscv.best_params_

Fitting 2 folds for each of 18 candidates, totalling 36 fits
[CV] END ....learning_rate=0.2, max_depth=9, n_estimators=30; total time= 2.6min
[CV] END ....learning_rate=0.2, max_depth=9, n_estimators=30; total time= 2.6min
[CV] END ....learning_rate=0.2, max_depth=9, n_estimators=50; total time= 4.3min
[CV] END ....learning_rate=0.2, max_depth=9, n_estimators=50; total time= 4.4min
[CV] END ....learning_rate=0.2, max_depth=9, n_estimators=70; total time= 6.0min
[CV] END ....learning_rate=0.2, max_depth=9, n_estimators=70; total time= 5.9min
[CV] END ...learning_rate=0.2, max_depth=12, n_estimators=30; total time= 3.7min
[CV] END ...learning_rate=0.2, max_depth=12, n_estimators=30; total time= 3.8min
[CV] END ...learning_rate=0.2, max_depth=12, n_estimators=50; total time= 6.5min
[CV] END ...learning_rate=0.2, max_depth=12, n_estimators=50; total time= 6.5min
[CV] END ...learning_rate=0.2, max_depth=12, n_estimators=70; total time= 9.0min
[CV] END ...learning_rate=0.2, max_depth=12, n_e

{'learning_rate': 0.2, 'max_depth': 14, 'n_estimators': 70}

In [19]:
xgb = XGBRegressor(random_state=21,max_depth=14,n_estimators=70,learning_rate=0.2)

In [20]:
xgb.fit(X_train,Y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.2, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=14, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=70, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=21, ...)

In [21]:
eval_fun(xgb,X_train,Y_train,Y_test,X_test)

r2_train_score ==> 0.8902930646633214 , mse_train_score ==> 2.017090262870099 , mae_train_score ==> 0.9430510768125687
r2_test_score ==> 0.8337398058112807 , mse_test_score ==> 3.0573009304849568 , mae_test_score ==> 1.1209945620620636


### Save The Model

In [22]:
pickle.dump(xgb, open('xgb_model.pkl','wb'))